# Prerequisites

1. I've created new `IAM` role on AWS
1. Added new permissions to access to DynamoDB
2. I've installed AWS CLI
3. Created own virtualenvironment and kernel
3. We need prepare our data



In [1]:
import os
import pandas as pd
import boto3
import hashlib
from IPython.core.interactiveshell import InteractiveShell
from settings import import_env_vars
InteractiveShell.ast_node_interactivity = 'all'
import_env_vars('../')

DATA_DIR = '../data/OctoberGPS/'

S3_BUCKET_NAME = os.environ['S3_BUCKET_NAME']
credentials = {
    'aws_access_key_id': os.environ['AWS_ACCESS_KEY_ID'],
    'aws_secret_access_key': os.environ['AWS_SECRET_ACCESS_KEY'],
    'region_name': os.environ['AWS_REGION']
}

## Notebook goal

Let's test next things:
1. **Connection using default credentials provided in `~/.aws/credentials`**
2. **Connection using external credentials**

As a connection test i'll send record and read status message

In [2]:
s3 = boto3.resource('s3',**credentials)

# PRINT bucket names hashed
for bucket in s3.buckets.all():
    print(hashlib.md5(bucket.name.encode()).hexdigest())

482323ab65a0cd3695a983e88ab6ca9d
79fa6431b018dc9774be08b4a501b56c
96c8be9723c2c8fd2ddd74b3ef1bb05a
579bac3f494a2b503a2c17d522ae11f9


Cool, connection is set up

## 1. Objects upload

In [3]:
%%time
FNAME = os.listdir(DATA_DIR)[0]
SAMPLE_FILE = DATA_DIR + FNAME

s3.Object(S3_BUCKET_NAME, 'GPS/' + FNAME).upload_file(Filename=SAMPLE_FILE)

CPU times: user 81 ms, sys: 8.33 ms, total: 89.3 ms
Wall time: 24.3 s


In [4]:
DATA_DIR = '../data/OctoberGPS/'

def upload_to_s3(credentials, bucket_name, bucket_fpath, local_fpath):
    s3 = boto3.resource('s3',**credentials)
    s3.Object(bucket_name, bucket_fpath).upload_file(Filename=local_fpath)


def upload_files_to_S3(local_fpaths, bucket_name, bucket_folder_path='',
                       aws_access_key_id=None,
                       aws_secret_access_key=None,
                       region_name=None):
    """
        Uploads your files to s3
        
        Arguments:
            * local_fpaths (list<str>): Path to files in local
    """
    # 1. Credentials generation
    credentials = {}
    if aws_access_key_id is not None: credentials['aws_access_key_id'] = aws_access_key_id
    if aws_secret_access_key is not None: credentials['aws_secret_access_key'] = aws_secret_access_key
    if region_name is not None: credentials['region_name'] = region_name
    
    # 3. Relative path in bucket
    fnames = list(map(lambda x: x.rsplit('/', 1)[1], local_fpaths))
    s3_fpaths = list(map(lambda x,y: x+y, [bucket_folder_path]*len(files), fnames))

    for local_file, bucket_file in zip(local_fpaths, s3_fpaths):
        upload_to_s3(credentials, bucket_name, bucket_file, local_file)

In [5]:
%%time
files = os.listdir(DATA_DIR)[:2]
local_fpaths = list(map(lambda x,y: x+y, [DATA_DIR]*len(files), files))


upload_files_to_S3(local_fpaths, S3_BUCKET_NAME, 'GPS/')

CPU times: user 873 µs, sys: 302 µs, total: 1.17 ms
Wall time: 563 µs


## 2. Listing files

In [11]:
pythonusecase = s3.Bucket(name = S3_BUCKET_NAME)

for files in pythonusecase.objects.all():
    print(hashlib.md5(files.key.encode()).hexdigest())

7d0999ecec4267ffa56a1c251de207c9
839590c7872d53d433cbba8bf57e5a39
bd2655a7a166d57dbf087f8c0f2b496b
da6f973b11561366ed9179887e70daaa
b33b1a4c81cd1312e0c44c0dbe1f657e
d02c2c0e35659ad18c8b988c9382eade
f05274fe1d9dde54bdd8b121dd486a38


## 3. Download files

In [ ]:
%%time
s3.Object(S3_BUCKET_NAME, FNAME).download_file(SAMPLE_FILE)

## 4. Delete object